In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch



In [6]:
import torch



In [ ]:
# 文件路径
file_path = 'test.ft.txt'

# 读取文本文件
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [8]:
git branch

SyntaxError: invalid syntax (3527392052.py, line 1)

In [7]:


# 存储标签和文本
labels = []
texts = []

# 解析每一行数据
for line in lines:
    label = int(line.split()[0].replace('label_', ''))  # 提取标签
    text = ' '.join(line.split()[1:])  # 提取文本
    labels.append(label)
    texts.append(text)

# 将数据转换为 DataFrame 以方便处理
df = pd.DataFrame({
    'label': labels,
    'text': texts
})

# 查看数据
print(df.head())

ValueError: invalid literal for int() with base 10: '___2'

In [ ]:
# Preprocess the data
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})  # Example mapping
train_texts, val_texts, train_labels, val_labels = train_test_split(df['reviewText'], df['sentiment'], test_size=0.2)

In [ ]:

# Preprocess the data
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})  # Example mapping
train_texts, val_texts, train_labels, val_labels = train_test_split(df['reviewText'], df['sentiment'], test_size=0.2)

# Load the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Tokenize the input texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# Create a dataset class
class AmazonReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = AmazonReviewsDataset(train_encodings, train_labels.tolist())
val_dataset = AmazonReviewsDataset(val_encodings, val_labels.tolist())

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()